## 필수 함수

In [61]:
#엑셀 파일 불러오는 함수
import pandas as pd

def load_and_clean_data(filepath, header):
    df = pd.read_excel(filepath, header = header)

    new_columns = []
    for col in df.columns:
        merged_name = [str(c) for c in col if 'Unnamed' not in str(c)]
        new_columns.append(' '.join(merged_name).strip())

    df.columns = new_columns
    df= df.fillna(method='ffill', axis=0)

    return df

In [62]:
def clean_major_name(df, col):
    df[col] = df[col].str.replace('·', '', regex=False)
    df[col] = df[col].str.replace('・', '', regex=False)
    df[col] = df[col].str.replace('?', '', regex=False)  # 그냥 없애기
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace(' ', '', regex=False)  # 띄어쓰기 제거
    return df

## 신입생 충원율 및 경쟁률

In [190]:
df_freshman = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\신입생충원율,신입생경쟁률\2023년 _대학_4-다. 신입생 충원 현황_학과별자료.xlsx", header=[3,4,5])
df_freshman = df_freshman[df_freshman['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [191]:
df_freshman.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과\n(모집단위),구분,학과특성,...,지원자 계,지원자 정원내\n(C),지원자 정원외,입학자 계,입학자 정원내(D) 남,입학자 정원내(D) 여,입학자 정원외 남,입학자 정원외 여,정원내\n신입생 충원율(%)\n(D/B) × 100,경쟁률\n(C/B)
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,1665,1486,179,186,41,124,5,16,100.0,9.0
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,물리치료학과,주간,일반과정,...,670,658,12,56,37,16,2,1,100.0,12.4
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,방사선학과,주간,일반과정,...,338,338,0,46,35,11,0,0,100.0,7.3
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,사회복지상담학과,주간,일반과정,...,114,114,0,13,7,6,0,0,50.0,4.4
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,스포츠재활복지학부,주간,일반과정,...,198,198,0,35,32,3,0,0,63.6,3.6


In [192]:
#기준연도 정수로 변경
df_freshman['기준연도'] = df_freshman['기준연도'].astype(int)

#야간 제외
df_freshman_noon = df_freshman[~df_freshman['구분'].str.contains('야간', na=False)]

In [193]:
def create_summarized_dataframe_for_freshman(df_gw):
    # 필요한 컬럼만 선택
    summary_df = df_gw[['기준연도','학교', '단과대학','학과\n(모집단위)','정원내\n신입생 충원율(%)\n(D/B) × 100','경쟁률\n(C/B)']].copy()
    return summary_df

df_freshman_selected = create_summarized_dataframe_for_freshman(df_freshman_noon)

In [194]:
df_freshman_selected.rename(columns={'학과\n(모집단위)': '학과'}, inplace=True)
df_freshman_selected.rename(columns={'정원내\n신입생 충원율(%)\n(D/B) × 100': '신입생 충원율(%)'}, inplace=True)
df_freshman_selected.rename(columns={'경쟁률\n(C/B)': '신입생 경쟁률'}, inplace=True)

In [195]:
df_freshman_selected

,기준연도,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률
0,2023,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0
1,2023,가야대학교(김해),단과대구분없음,물리치료학과,100.0,12.4
2,2023,가야대학교(김해),단과대구분없음,방사선학과,100.0,7.3
3,2023,가야대학교(김해),단과대구분없음,사회복지상담학과,50.0,4.4
4,2023,가야대학교(김해),단과대구분없음,스포츠재활복지학부,63.6,3.6
...,...,...,...,...,...,...
7529,2023,화성의과학대학교,단과대구분없음,뷰티아트케어학과,47.8,12.7
7530,2023,화성의과학대학교,단과대구분없음,사회복지보건학과,39.1,4.8
7531,2023,화성의과학대학교,단과대구분없음,스포츠건강학부,29.3,2.5
7532,2023,화성의과학대학교,단과대구분없음,의생명과학과,100.0,5.9


## 재학생충원율

In [197]:
#재학생충원율 데이터 불러오기
df_student_2022_sec_and_2023_first = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\재학생충원율\2023년 _대학_4-라-1. 재학생 충원율_학과별자료.xlsx", header=[3,4])
df_student_2022_sec_and_2023_first = df_student_2022_sec_and_2023_first[df_student_2022_sec_and_2023_first['학교종류'] == '대학교']
df_student_2023_sec_and_2024_first = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\재학생충원율\2024년 _대학_4-라-1. 재학생 충원율_학과별자료.xlsx", header=[3,4])
df_student_2023_sec_and_2024_first = df_student_2023_sec_and_2024_first[df_student_2023_sec_and_2024_first['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [198]:
#기준연도 2023 상반기, 하반기만 선택
df_student_2023_first = df_student_2022_sec_and_2023_first[df_student_2022_sec_and_2023_first['기준연도'] == '2023 년 상반기']
df_student_2023_sec = df_student_2023_sec_and_2024_first[df_student_2023_sec_and_2024_first['기준연도'] == '2023 년 하반기']

In [199]:
def create_summarized_dataframe(df):
    # 필요한 컬럼만 선택
    summary_df = df[['기준연도','학교', '계열','재학생충원율(%){C/(A-B)}X100','정원내 재학생 충원율(%){D/(A-B)}X100']].copy()
    return summary_df

df_student_2023_first_filtered = create_summarized_dataframe(df_student_2023_first)
df_student_2023_sec_filtered = create_summarized_dataframe(df_student_2023_sec)

In [200]:
# 두 DataFrame을 수직 결합
df_all = pd.concat([df_student_2023_first_filtered, df_student_2023_sec_filtered], ignore_index=True)

# 기준연도에서 연도(숫자)만 추출
df_all['연도'] = df_all['기준연도'].str.extract(r'(\d{4})')

In [201]:
# 연도 및 학교 기준으로 평균 계산
df_avg = df_all.groupby(['연도', '학교','계열'])[
    ['재학생충원율(%){C/(A-B)}X100', '정원내 재학생 충원율(%){D/(A-B)}X100']
].mean().reset_index()

In [202]:
df_avg

,연도,학교,계열,재학생충원율(%){C/(A-B)}X100,정원내 재학생 충원율(%){D/(A-B)}X100
0,2023,가야대학교(고령) _제2캠퍼스,공학계열,0.0,0.0
1,2023,가야대학교(고령) _제2캠퍼스,예ㆍ체능계열,0.0,0.0
2,2023,가야대학교(고령) _제2캠퍼스,의학계열,0.0,0.0
3,2023,가야대학교(고령) _제2캠퍼스,인문ㆍ사회계열,0.0,0.0
4,2023,가야대학교(고령) _제2캠퍼스,자연과학계열,0.0,0.0
...,...,...,...,...,...
1070,2023,화성의과학대학교,공학계열,0.0,0.0
1071,2023,화성의과학대학교,예ㆍ체능계열,22.8,22.8
1072,2023,화성의과학대학교,의학계열,0.0,0.0
1073,2023,화성의과학대학교,인문ㆍ사회계열,27.3,27.3


## 중도탈락률

In [204]:
df_dropout = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\중도탈락률\2024년 _대학_4-사. 중도탈락 학생 현황_학과별자료.xlsx", header=[3,4])
df_dropout = df_dropout[df_dropout['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [205]:
df_dropout.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과\n(전공),구분,학과특성,...,사유별 중도탈락 학생 계(B),사유별 중도탈락 학생 미등록,사유별 중도탈락 학생 미복학,사유별 중도탈락 학생 자퇴,사유별 중도탈락 학생 학사경고,사유별 중도탈락 학생 학생활동,사유별 중도탈락 학생 유급제적,사유별 중도탈락 학생 재학연한초과,사유별 중도탈락 학생 기타,중도탈락학생비율(%)\n(B/A) × 100
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,36,2,3,31,0,0,0,0,0,4.5
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경영물류학과,주간,일반과정,...,8,1,1,6,0,0,0,0,0,16.7
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰소방학과,주간,일반과정,...,29,2,9,18,0,0,0,0,0,31.9
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,귀금속주얼리학과,주간,일반과정,...,2,0,1,1,0,0,0,0,0,8.7
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,물리치료학과,주간,일반과정,...,16,3,1,12,0,0,0,0,0,5.6


In [206]:
#기준연도 정수로 변경
df_dropout['기준연도'] = df_dropout['기준연도'].astype(int)

#야간 제외
df_dropout_noon = df_dropout[~df_dropout['구분'].str.contains('야간', na=False)]

In [207]:
df_dropout_selected = df_dropout_noon[['기준연도','학교','단과대학','학과\n(전공)','중도탈락학생비율(%)\n(B/A) × 100']]

In [208]:
df_dropout_selected.rename(columns = {'학과\n(전공)':'학과'}, inplace = True)
df_dropout_selected.rename(columns = {'중도탈락학생비율(%)\n(B/A) × 100':'중도탈락률(%)'}, inplace = True)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\2840908820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropout_selected.rename(columns = {'학과\n(전공)':'학과'}, inplace = True)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\2840908820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropout_selected.rename(columns = {'중도탈락학생비율(%)\n(B/A) × 100':'중도탈락률(%)'}, inplace = True)


In [209]:
df_dropout_selected

,기준연도,학교,단과대학,학과,중도탈락률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,4.5
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,16.7
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.9
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,8.7
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,5.6
...,...,...,...,...,...
13484,2023,화성의과학대학교,단과대구분없음,스포츠건강학부,0.0
13485,2023,화성의과학대학교,단과대구분없음,스포츠레저학과,26.1
13486,2023,화성의과학대학교,단과대구분없음,의생명과학과,36.4
13487,2023,화성의과학대학교,단과대구분없음,컴퓨터사이언스학과,26.7


## 졸업생 취업률 및 진학률

In [211]:
df_gradu = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\졸업생 취업률 진학률\2024년 _대학_5-다. 졸업생의 취업 현황_학과별자료.xlsx", header=[3,4,5,6,7])
df_gradu = df_gradu[df_gradu['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [212]:
df_gradu.head()

,기준연도,학교종류,설립구분,지역,상태,학교명,단과대학,학과(전공),구분,학과특성,...,유지\n취업률 1차 여,유지\n취업률 2차 계,유지\n취업률 2차 남,유지\n취업률 2차 여,유지\n취업률 3차 계,유지\n취업률 3차 남,유지\n취업률 3차 여,유지\n취업률 4차 계,유지\n취업률 4차 남,유지\n취업률 4차 여
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,95.9,90.1,82.1,91.9,87.4,71.4,91.1,84.1,71.4,87.0
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경영물류학과,주간,일반과정,...,0.0,89.5,94.4,0.0,78.9,83.3,0.0,78.9,83.3,0.0
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰소방학과,주간,일반과정,...,100.0,80.0,75.0,100.0,80.0,75.0,100.0,60.0,75.0,0.0
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,귀금속주얼리학과,주간,일반과정,...,100.0,100.0,100.0,100.0,90.0,83.3,100.0,70.0,66.7,75.0
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,물리치료학과,주간,일반과정,...,81.8,84.8,95.8,72.7,84.8,95.8,72.7,84.8,95.8,72.7


In [213]:
#기준연도 정수로 변경
df_gradu['기준연도'] = df_gradu['기준연도'].astype(int)

#야간 제외
df_gradu_noon = df_gradu[~df_gradu['구분'].str.contains('야간', na=False)]

In [214]:
df_gradu_noon.columns

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교명', '단과대학', '학과(전공)', '구분',
       '학과특성', '졸업자(A) 남', '졸업자(A) 여', '취업자(B) 건강보험\n직장가입자 남',
       '취업자(B) 건강보험\n직장가입자 여', '취업자(B) 해외\n취업자 남', '취업자(B) 해외\n취업자 여',
       '취업자(B) 농림어업\n종사자 남', '취업자(B) 농림어업\n종사자 여', '취업자(B) 개인창작\n활동\n종사자 남',
       '취업자(B) 개인창작\n활동\n종사자 여', '취업자(B) 1인\n창(사)업자 남', '취업자(B) 1인\n창(사)업자 여',
       '취업자(B) 프리랜서 남', '취업자(B) 프리랜서 여', '진학자(C) 남', '진학자(C) 여', '입대자(D)',
       '취업불가능자(E) 남', '취업불가능자(E) 여', '외국인유학생(F) 남', '외국인유학생(F) 여',
       '제외인정자(G) 남', '제외인정자(G) 여', '기타 남', '기타 여', '미상 남', '미상 여',
       '취업률(%)\n[B/{A-(C+D+E+F+G)x100', '입학\n당시\n기취업자 남', '입학\n당시\n기취업자 여',
       '교내취업자 남', '교내취업자 여', '유지\n취업률 1차 계', '유지\n취업률 1차 남', '유지\n취업률 1차 여',
       '유지\n취업률 2차 계', '유지\n취업률 2차 남', '유지\n취업률 2차 여', '유지\n취업률 3차 계',
       '유지\n취업률 3차 남', '유지\n취업률 3차 여', '유지\n취업률 4차 계', '유지\n취업률 4차 남',
       '유지\n취업률 4차 여'],
      dtype='object')

In [215]:
#필요한 열 선택
df_gradu_selected = df_gradu_noon[['기준연도', '학교명', '단과대학', '학과(전공)', '졸업자(A) 남', '졸업자(A) 여', '취업률(%)\n[B/{A-(C+D+E+F+G)x100',  '진학자(C) 남', '진학자(C) 여']].copy()

In [216]:
#졸업자 수, 진학자 수, 진학률 계산
df_gradu_selected['졸업자 수'] = df_gradu_selected['졸업자(A) 남'] + df_gradu_selected['졸업자(A) 여']
df_gradu_selected['진학자 수'] = df_gradu_selected['진학자(C) 남'] + df_gradu_selected['진학자(C) 여']
df_gradu_selected['졸업생 진학률(%)'] = (df_gradu_selected['진학자 수'] / df_gradu_selected['졸업자 수'] * 100).round(2)

In [217]:
df_gradu_selected.rename(columns = {'학교명' : '학교'}, inplace = True)
df_gradu_selected.rename(columns = {'학과(전공)' : '학과'}, inplace = True)
df_gradu_selected.rename(columns = {'취업률(%)\n[B/{A-(C+D+E+F+G)x100' : '졸업생 취업률(%)'}, inplace = True)

In [218]:
#최종 필요한 열 선택
df_gradu_final = df_gradu_selected[['기준연도','학교','단과대학','학과','졸업생 취업률(%)','졸업생 진학률(%)']].copy()

In [219]:
df_gradu_final

,기준연도,학교,단과대학,학과,졸업생 취업률(%),졸업생 진학률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,85.9,0.0
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,70.4,0.0
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.3,0.0
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,57.9,0.0
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,88.7,0.0
...,...,...,...,...,...,...
9908,2023,화성의과학대학교,단과대구분없음,뷰티디자인학과,71.4,0.0
9909,2023,화성의과학대학교,단과대구분없음,사회복지학과,38.5,0.0
9910,2023,화성의과학대학교,단과대구분없음,생명과학과,0.0,0.0
9911,2023,화성의과학대학교,단과대구분없음,스포츠레저학과,100.0,12.5


## 교원 1인당 연구실적
### 국내 / 국제 / 저역서

In [220]:
df_research = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\교원 1인당 연구실적\전임교원의 연구 실적 (대학)_2025-07-03133559666.xlsx", header=[3,4,5,6,7])
df_research = df_research[df_research['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [221]:
df_research.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과,전임\n교원 계,전임\n교원 남,...,전임교원 1인당 논문 실적 국제\n기준 여,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 계,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 남,전임교원 1인당 논문 실적 연구재단등재지(후보포함) 여,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 계,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 남,전임교원 1인당 논문 실적 SCI급/SCOPUS학술지 여,전임교원 1인당 저∙역서 실적 계,전임교원 1인당 저∙역서 실적 남,전임교원 1인당 저∙역서 실적 여
0,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,방사선학과,6,5,...,0.0000,0.7500,0.5200,1.9000,0.0000,0.0,0.0000,0.1833,0.22,0.0000
1,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,귀금속주얼리학과,3,3,...,0.0000,0.6667,0.6667,0.0000,0.0000,0.0,0.0000,0.0000,0.00,0.0000
2,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,교양학부,6,4,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.00,0.0000
3,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,20,1,...,0.0624,0.5446,0.0000,0.5733,0.0393,0.0,0.0414,0.0867,0.00,0.0912
4,2023,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,안경광학과,1,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,1.0000,1.00,0.0000


In [222]:
df_research.columns

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '단과대학', '학과', '전임\n교원 계',
       '전임\n교원 남', '전임\n교원 여', '논문 실적 총계 계', '논문 실적 총계 남', '논문 실적 총계 여',
       '논문 실적 국내 소계 계', '논문 실적 국내 소계 남', '논문 실적 국내 소계 여',
       '논문 실적 국내 연구재단\n등재지\n(후보포함) 계', '논문 실적 국내 연구재단\n등재지\n(후보포함) 남',
       '논문 실적 국내 연구재단\n등재지\n(후보포함) 여', '논문 실적 국내 기타\n국내발간\n일반학술지 계',
       '논문 실적 국내 기타\n국내발간\n일반학술지 남', '논문 실적 국내 기타\n국내발간\n일반학술지 여',
       '논문 실적 국제 소계 계', '논문 실적 국제 소계 남', '논문 실적 국제 소계 여',
       '논문 실적 국제 SCI급\n/SCOPUS\n학술지 계', '논문 실적 국제 SCI급\n/SCOPUS\n학술지 남',
       '논문 실적 국제 SCI급\n/SCOPUS\n학술지 여', '논문 실적 국제 기타\n국제발간\n일반학술지 계',
       '논문 실적 국제 기타\n국제발간\n일반학술지 남', '논문 실적 국제 기타\n국제발간\n일반학술지 여',
       '저∙역서 실적 계 계', '저∙역서 실적 계 남', '저∙역서 실적 계 여', '저∙역서 실적 저서 계',
       '저∙역서 실적 저서 남', '저∙역서 실적 저서 여', '저∙역서 실적 역서 계', '저∙역서 실적 역서 남',
       '저∙역서 실적 역서 여', '전임교원 1인당 논문 실적 국내\n기준 계', '전임교원 1인당 논문 실적 국내\n기준 남',
       '전임교원 1인당 논문 실적 국내\n기준 여', '전임교원 1인당 논문 실적 국제\n기준 계',
       '전임교원 1인당 논문 실적 국제\n기준 남', '전임교원 1인

In [223]:
def create_summarized_dataframe(df_gw):
    # 필요한 컬럼만 선택
    summary_df = df_gw[['기준연도','학교', '단과대학', '학과','전임교원 1인당 논문 실적 국내\n기준 계',  '전임교원 1인당 논문 실적 국제\n기준 계','전임교원 1인당 저∙역서 실적 계']].copy()
    return summary_df

df_research_summary = create_summarized_dataframe(df_research)

In [224]:
#컬럼명 변경
df_research_summary.rename(columns = {'전임교원 1인당 논문 실적 국내\n기준 계' : '전임교원 1인당 논문 실적 국내'}, inplace = True)
df_research_summary.rename(columns = {'전임교원 1인당 논문 실적 국제\n기준 계' : '전임교원 1인당 논문 실적 국제'}, inplace = True)
df_research_summary.rename(columns = {'전임교원 1인당 저∙역서 실적 계' : '전임교원 1인당 저∙역서 실적'}, inplace = True)

In [225]:
df_research_summary

,기준연도,학교,단과대학,학과,전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적
0,2023,가야대학교(김해),단과대구분없음,방사선학과,0.7500,0.0000,0.1833
1,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,0.6667,0.0000,0.0000
2,2023,가야대학교(김해),단과대구분없음,교양학부,0.0000,0.0000,0.0000
3,2023,가야대학교(김해),단과대구분없음,간호학과,0.5446,0.0593,0.0867
4,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0000,0.0000,1.0000
...,...,...,...,...,...,...,...
9161,2023,화성의과학대학교,단과대구분없음,의생명과학과,0.0000,0.6416,0.0000
9162,2023,화성의과학대학교,단과대구분없음,서비스디자인학과,0.0000,0.0000,0.0000
9163,2023,화성의과학대학교,단과대구분없음,의료심리학과,1.1000,0.0000,0.0000
9164,2023,화성의과학대학교,단과대구분없음,바이오헬스케어학과,0.0000,0.9472,0.3000


## 교원 1인당 연구비 수혜 실적
### 교내 / 교외

In [226]:
df_research_money = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\교원1인당 연구비\2024년 _대학_12-가. 연구비 수혜 실적_학과별자료.xlsx", header=[3,4,5,6,7])
df_research_money = df_research_money[df_research_money['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [227]:
df_research_money.columns

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '단과대학', '학과', '전임\n교원\n수 남',
       '전임\n교원\n수 여', '연구비 지원 계 과제수 남', '연구비 지원 계 과제수 여', '연구비 지원 계 연구비 남',
       '연구비 지원 계 연구비 여', '연구비 지원 교내 과제수 남', '연구비 지원 교내 과제수 여',
       '연구비 지원 교내 연구비 남', '연구비 지원 교내 연구비 여', '연구비 지원 교외 중앙정부 과제수 남',
       '연구비 지원 교외 중앙정부 과제수 여', '연구비 지원 교외 중앙정부 연구비 남', '연구비 지원 교외 중앙정부 연구비 여',
       '연구비 지원 교외 지자체 과제수 남', '연구비 지원 교외 지자체 과제수 여', '연구비 지원 교외 지자체 연구비 남',
       '연구비 지원 교외 지자체 연구비 여', '연구비 지원 교외 민간 과제수 남', '연구비 지원 교외 민간 과제수 여',
       '연구비 지원 교외 민간 연구비 남', '연구비 지원 교외 민간 연구비 여', '연구비 지원 교외 외국 과제수 남',
       '연구비 지원 교외 외국 과제수 여', '연구비 지원 교외 외국 연구비 남', '연구비 지원 교외 외국 연구비 여',
       '대응자금 교내 남', '대응자금 교내 여', '대응자금 교외 남', '대응자금 교외 여',
       '전임교원\n1인당\n연구비 교내 남', '전임교원\n1인당\n연구비 교내 여', '전임교원\n1인당\n연구비 교외 남',
       '전임교원\n1인당\n연구비 교외 여'],
      dtype='object')

In [228]:
df_research_money_summary = df_research_money[['기준연도','학교', '단과대학', '학과','전임\n교원\n수 남','전임\n교원\n수 여','연구비 지원 교내 연구비 남', '연구비 지원 교내 연구비 여',
                                                '연구비 지원 교외 중앙정부 연구비 남', '연구비 지원 교외 중앙정부 연구비 여','연구비 지원 교외 지자체 연구비 남','연구비 지원 교외 지자체 연구비 여',
                                                '연구비 지원 교외 민간 연구비 남', '연구비 지원 교외 민간 연구비 여','연구비 지원 교외 외국 연구비 남', '연구비 지원 교외 외국 연구비 여']].copy()

In [229]:
df_research_money_summary['기준연도'] = df_research_money_summary['기준연도'].astype(int)

In [230]:
df_research_money_summary.columns

Index(['기준연도', '학교', '단과대학', '학과', '전임\n교원\n수 남', '전임\n교원\n수 여',
       '연구비 지원 교내 연구비 남', '연구비 지원 교내 연구비 여', '연구비 지원 교외 중앙정부 연구비 남',
       '연구비 지원 교외 중앙정부 연구비 여', '연구비 지원 교외 지자체 연구비 남', '연구비 지원 교외 지자체 연구비 여',
       '연구비 지원 교외 민간 연구비 남', '연구비 지원 교외 민간 연구비 여', '연구비 지원 교외 외국 연구비 남',
       '연구비 지원 교외 외국 연구비 여'],
      dtype='object')

In [231]:
#교내연구비계산
df_research_money_summary['교내연구비'] = df_research_money_summary['연구비 지원 교내 연구비 남'] + df_research_money_summary['연구비 지원 교내 연구비 여'] 

#교외연구비계산
df_research_money_summary['교외연구비'] = df_research_money_summary['연구비 지원 교외 중앙정부 연구비 남'] + df_research_money_summary['연구비 지원 교외 중앙정부 연구비 여'] + \
    df_research_money_summary['연구비 지원 교외 지자체 연구비 남'] + df_research_money_summary['연구비 지원 교외 지자체 연구비 여'] + \
    df_research_money_summary['연구비 지원 교외 민간 연구비 남'] + df_research_money_summary['연구비 지원 교외 민간 연구비 여'] +\
    df_research_money_summary['연구비 지원 교외 외국 연구비 남'] + df_research_money_summary['연구비 지원 교외 외국 연구비 여']

#총교원수 계산
df_research_money_summary['총교원수'] = df_research_money_summary['전임\n교원\n수 남'] + df_research_money_summary['전임\n교원\n수 여']


#1인당 연구비 계산
df_research_money_summary['교내 1인당 연구비(천원)'] = (df_research_money_summary['교내연구비'] / df_research_money_summary['총교원수']).round(2)
df_research_money_summary['교외 1인당 연구비(천원)'] = (df_research_money_summary['교외연구비'] / df_research_money_summary['총교원수']).round(2)


In [232]:
df_research_money_summary.head(6)

,기준연도,학교,단과대학,학과,전임\n교원\n수 남,전임\n교원\n수 여,연구비 지원 교내 연구비 남,연구비 지원 교내 연구비 여,연구비 지원 교외 중앙정부 연구비 남,연구비 지원 교외 중앙정부 연구비 여,...,연구비 지원 교외 지자체 연구비 여,연구비 지원 교외 민간 연구비 남,연구비 지원 교외 민간 연구비 여,연구비 지원 교외 외국 연구비 남,연구비 지원 교외 외국 연구비 여,교내연구비,교외연구비,총교원수,교내 1인당 연구비(천원),교외 1인당 연구비(천원)
0,2023,가야대학교(김해),단과대구분없음,방사선학과,5,1,5000,2500,0,0,...,0,0,0,0,0,7500,0,6,1250.00,0.00
1,2023,가야대학교(김해),단과대구분없음,안경광학과,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.00,0.00
2,2023,가야대학교(김해),단과대구분없음,간호학과,1,19,2500,15500,0,0,...,0,0,0,0,0,18000,0,20,900.00,0.00
3,2023,가야대학교(김해),단과대구분없음,교양학부,4,2,0,0,0,0,...,0,0,0,0,0,0,0,6,0.00,0.00
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,3,0,4500,0,0,0,...,0,0,0,0,0,4500,0,3,1500.00,0.00
5,2023,가야대학교(김해),단과대구분없음,물리치료학과,5,2,9500,0,28224,25026,...,0,0,0,0,0,9500,53250,7,1357.14,7607.14


In [233]:
df_research_money_final = df_research_money_summary[['기준연도', '학교', '단과대학', '학과', '교내 1인당 연구비(천원)','교외 1인당 연구비(천원)' ]].copy()

In [234]:
df_research_money_final.head(10)

,기준연도,학교,단과대학,학과,교내 1인당 연구비(천원),교외 1인당 연구비(천원)
0,2023,가야대학교(김해),단과대구분없음,방사선학과,1250.00,0.00
1,2023,가야대학교(김해),단과대구분없음,안경광학과,0.00,0.00
2,2023,가야대학교(김해),단과대구분없음,간호학과,900.00,0.00
3,2023,가야대학교(김해),단과대구분없음,교양학부,0.00,0.00
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,1500.00,0.00
5,2023,가야대학교(김해),단과대구분없음,물리치료학과,1357.14,7607.14
6,2023,가야대학교(김해),단과대구분없음,유아교육과,833.33,0.00
7,2023,가야대학교(김해),단과대구분없음,부동산금융재테크학과,2000.00,0.00
8,2023,가야대학교(김해),단과대구분없음,작업치료학과,2500.00,0.00
9,2023,가야대학교(김해),단과대구분없음,사회복지상담학과,833.33,0.00


## 캡스톤디자인운영현황

In [238]:
df_caps = pd.read_excel(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\캡스톤디자인\20250915_12-카-2 캡스톤 디자인 운영 현황_대학(2023~2025).xlsx", sheet_name= '2024')
df_caps = df_caps[df_caps['학교종류명'] == '대학교']

In [239]:
df_caps.columns

Index(['기준년도', '학교종류명', '학교설립구분명', '공시지역명', '학교상태명', '학교명', '본분교구분명', '학과명',
       '계열대분류명', '계열중분류명', '계열소분류명', '자체계열명', '단과대학명', '주야간구분명', '학기구분명',
       '학점수', '이수학생수_해당학과', '이수학생수_타학과 계열_인문사회', '이수학생수_타학과 계열_자연과학',
       '이수학생수_타학과 계열_공학', '이수학생수_타학과 계열_의학', '이수학생수_타학과 계열_예체능', '지원금수령학생수',
       '지원금수령총금액_시제(작)품 제작비', '지원금수령총금액_교육프로그램 지원비'],
      dtype='object')

In [240]:
#야간 제외
df_caps_noon = df_caps[df_caps['주야간구분명'] != '야간']

#학교명에 본분교구분넣기
df_caps_noon["학교"] = df_caps_noon.apply(
    lambda x: x["학교명"] if x["본분교구분명"] == "본교" 
    else f"{x['학교명']}_{x['본분교구분명']}",
    axis=1
)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\340856512.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_caps_noon["학교"] = df_caps_noon.apply(


In [241]:
#운영현황 구하기
df_caps_noon['캡스톤디자인이수학생수'] = df_caps_noon['이수학생수_해당학과'] + df_caps_noon['이수학생수_타학과 계열_인문사회'] + df_caps_noon['이수학생수_타학과 계열_자연과학'] + df_caps_noon['이수학생수_타학과 계열_공학'] \
                                        + df_caps_noon['이수학생수_타학과 계열_의학'] + df_caps_noon['이수학생수_타학과 계열_예체능']

df_caps_avg = df_caps_noon.groupby(['학교','학과명'], as_index= False)['캡스톤디자인이수학생수'].mean()

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\2585849525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_caps_noon['캡스톤디자인이수학생수'] = df_caps_noon['이수학생수_해당학과'] + df_caps_noon['이수학생수_타학과 계열_인문사회'] + df_caps_noon['이수학생수_타학과 계열_자연과학'] + df_caps_noon['이수학생수_타학과 계열_공학'] \


In [242]:
#컬럼명 변경
df_caps_avg.rename(columns = {'학과명' : '학과'}, inplace = True)

## 현장실습 이수학생 비율

In [245]:
df_intern = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\지표new-6페이지\2024년 _대학_12-카-1. 현장실습 운영 현황_학과별자료.xlsx", header = [3,4,5,6])
df_intern = df_intern[df_intern['학교종류'] == '대학교']
df_intern_deno = load_and_clean_data(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\지표new-6페이지\2023년 _대학_4-마. 재적 학생 현황_학과별자료.xlsx", header = [3,4,5])
df_intern_deno = df_intern_deno[df_intern_deno['학교종류'] == '대학교']

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3884397518.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [246]:
#기준연도 정수로 변경
df_intern['기준연도'] = df_intern['기준연도'].astype(int)

#야간 제외하기
df_intern_noon = df_intern[df_intern['구분'] != '야간']

In [247]:
#필요한 열 선택
df_intern_selected = df_intern_noon[['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '단과대학', '학과(전공)', '구분','실습기간 4주 이상 이수학생 수','실습기간 8주 이상 이수학생 수','실습기간 12주 이상 이수학생 수']]

In [248]:
df_intern_selected['현장실습 이수학생 수 곱셈'] = df_intern_selected['실습기간 4주 이상 이수학생 수'] + df_intern_selected['실습기간 8주 이상 이수학생 수']*2 + df_intern_selected['실습기간 12주 이상 이수학생 수']*3

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\2147402955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intern_selected['현장실습 이수학생 수 곱셈'] = df_intern_selected['실습기간 4주 이상 이수학생 수'] + df_intern_selected['실습기간 8주 이상 이수학생 수']*2 + df_intern_selected['실습기간 12주 이상 이수학생 수']*3


In [249]:
df_intern_selected

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과(전공),구분,실습기간 4주 이상 이수학생 수,실습기간 8주 이상 이수학생 수,실습기간 12주 이상 이수학생 수,현장실습 이수학생 수 곱셈
0,2023,대학교,사립,경기,기존,가천대학교,간호대학,간호학과,주간,0,0,0,0
3,2023,대학교,사립,경기,기존,가천대학교,경영대학,경영학과,주간,0,0,0,0
4,2023,대학교,사립,경기,기존,가천대학교,경영대학,경영학과(경영학트랙),주간,0,0,0,0
6,2023,대학교,사립,경기,기존,가천대학교,경영대학,경영학과(글로벌경영학트랙),주간,0,0,0,0
7,2023,대학교,사립,경기,기존,가천대학교,경영대학,경영학부,주간,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2023,대학교,사립,세종,기존,홍익대학교 _제2캠퍼스,조형대학,디자인·영상학부 디지털미디어디자인전공,주간,0,1,1,5
3995,2023,대학교,사립,세종,기존,홍익대학교 _제2캠퍼스,조형대학,디자인컨버전스학부,주간,2,14,6,48
3996,2023,대학교,사립,세종,기존,홍익대학교 _제2캠퍼스,조형대학,영상ㆍ애니메이션학부,주간,1,4,5,24
3997,2023,대학교,사립,세종,기존,홍익대학교 _제2캠퍼스,단과대구분없음,게임학부 게임그래픽디자인전공(미술계),주간,0,0,3,9


In [250]:
df_intern_deno.head()

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과\n(전공),구분,학과특성,...,학사학위취득유예학생(C) 남 정원내,학사학위취득유예학생(C) 남 정원외,학사학위취득유예학생(C) 여 정원내,학사학위취득유예학생(C) 여 정원외,재적학생(D=A+B+C) 계 정원내,재적학생(D=A+B+C) 계 정원외,재적학생(D=A+B+C) 남 정원내,재적학생(D=A+B+C) 남 정원외,재적학생(D=A+B+C) 여 정원내,재적학생(D=A+B+C) 여 정원외
0,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,방사선학과,주간,일반과정,...,0,0,0,0,225,8,157,6,68,2
1,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,간호학과,주간,일반과정,...,0,0,0,0,666,133,226,35,440,98
2,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경영물류학과,주간,일반과정,...,0,0,0,0,45,3,42,1,3,2
3,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰소방학과,주간,일반과정,...,0,0,0,0,91,0,82,0,9,0
4,2023.0,대학교,사립,경남,기존,가야대학교(김해),단과대구분없음,경찰행정학과,주간,일반과정,...,0,0,0,0,0,0,0,0,0,0


In [251]:
df_intern_deno.columns

Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '단과대학', '학과\n(전공)', '구분',
       '학과특성', '학과상태', '계열', '학생정원', '재학생(A) 계 정원내', '재학생(A) 계 정원외',
       '재학생(A) 남 정원내', '재학생(A) 남 정원외', '재학생(A) 여 정원내', '재학생(A) 여 정원외',
       '휴학생(B) 계 정원내', '휴학생(B) 계 정원외', '휴학생(B) 남 정원내', '휴학생(B) 남 정원외',
       '휴학생(B) 여 정원내', '휴학생(B) 여 정원외', '학사학위취득유예학생(C) 계 정원내',
       '학사학위취득유예학생(C) 계 정원외', '학사학위취득유예학생(C) 남 정원내', '학사학위취득유예학생(C) 남 정원외',
       '학사학위취득유예학생(C) 여 정원내', '학사학위취득유예학생(C) 여 정원외', '재적학생(D=A+B+C) 계 정원내',
       '재적학생(D=A+B+C) 계 정원외', '재적학생(D=A+B+C) 남 정원내', '재적학생(D=A+B+C) 남 정원외',
       '재적학생(D=A+B+C) 여 정원내', '재적학생(D=A+B+C) 여 정원외'],
      dtype='object')

In [252]:
#기준연도 정수로 변경
df_intern_deno['기준연도'] = df_intern_deno['기준연도'].astype(int)

#야간 제외
df_intern_deno_noon = df_intern_deno[df_intern_deno['구분'] != '야간']

In [253]:
df_intern_deno_selected = df_intern_deno_noon[['지역', '학교', '단과대학', '학과\n(전공)', '구분','학과상태', '계열', '학생정원']].copy()

In [254]:
df_intern_deno_selected

,지역,학교,단과대학,학과\n(전공),구분,학과상태,계열,학생정원
0,경남,가야대학교(김해),단과대구분없음,방사선학과,주간,기존,자연과학계열,180
1,경남,가야대학교(김해),단과대구분없음,간호학과,주간,기존,자연과학계열,602
2,경남,가야대학교(김해),단과대구분없음,경영물류학과,주간,폐과,인문ㆍ사회계열,65
3,경남,가야대학교(김해),단과대구분없음,경찰소방학과,주간,폐과,인문ㆍ사회계열,30
4,경남,가야대학교(김해),단과대구분없음,경찰행정학과,주간,폐과,인문ㆍ사회계열,80
...,...,...,...,...,...,...,...,...
14426,경기,화성의과학대학교,단과대구분없음,스포츠건강학부,주간,변경,예ㆍ체능계열,40
14427,경기,화성의과학대학교,단과대구분없음,스포츠레저학과,주간,폐과,예ㆍ체능계열,140
14428,경기,화성의과학대학교,단과대구분없음,의생명과학과,주간,변경,자연과학계열,22
14429,경기,화성의과학대학교,단과대구분없음,컴퓨터학과,주간,폐과,자연과학계열,92


In [255]:
#야간 제외
df_intern_deno_selected_noon = df_intern_deno_selected[df_intern_deno_selected['구분'] != '야간']

In [256]:
#컬럼명 변경
df_intern_deno_selected_noon.rename(columns = {'학과\n(전공)' : '학과'}, inplace = True)
df_intern_selected.rename(columns = {'학과(전공)' : '학과'}, inplace = True)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\498558487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intern_selected.rename(columns = {'학과(전공)' : '학과'}, inplace = True)


In [257]:
#텍스트 정리
df_intern_deno_selected_clean = clean_major_name(df_intern_deno_selected_noon, '학과')
df_intern_selected_clean = clean_major_name(df_intern_selected, '학과')

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\635624862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace('·', '', regex=False)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\635624862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace('・', '', regex=False)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\635624862.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [258]:
df_intern_add = pd.merge(df_intern_selected_clean,df_intern_deno_selected_clean, how = 'outer',on = ['지역','학교','단과대학','학과','구분'])

In [259]:
df_intern_add

,기준연도,학교종류,설립구분,지역,상태,학교,단과대학,학과,구분,실습기간 4주 이상 이수학생 수,실습기간 8주 이상 이수학생 수,실습기간 12주 이상 이수학생 수,현장실습 이수학생 수 곱셈,학과상태,계열,학생정원
0,NaN,NaN,NaN,강원,NaN,가톨릭관동대학교,경영대학,경영정보학과,주간,NaN,NaN,NaN,NaN,폐과,인문ㆍ사회계열,0.0
1,NaN,NaN,NaN,강원,NaN,가톨릭관동대학교,경영대학,경영학과,주간,NaN,NaN,NaN,NaN,폐과,인문ㆍ사회계열,0.0
2,NaN,NaN,NaN,강원,NaN,가톨릭관동대학교,경영대학,경제금융학과,주간,NaN,NaN,NaN,NaN,폐과,인문ㆍ사회계열,0.0
3,NaN,NaN,NaN,강원,NaN,가톨릭관동대학교,경영대학,광고홍보학과,주간,NaN,NaN,NaN,NaN,폐과,인문ㆍ사회계열,0.0
4,NaN,NaN,NaN,강원,NaN,가톨릭관동대학교,경영대학,무역학과,주간,NaN,NaN,NaN,NaN,폐과,인문ㆍ사회계열,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13462,NaN,NaN,NaN,충북,NaN,한국교원대학교,제3대학,화학교육과,주간,NaN,NaN,NaN,NaN,기존,자연과학계열,77.0
13463,NaN,NaN,NaN,충북,NaN,한국교원대학교,제3대학,환경교육과,주간,NaN,NaN,NaN,NaN,기존,자연과학계열,76.0
13464,NaN,NaN,NaN,충북,NaN,한국교원대학교,제4대학,미술교육과,주간,NaN,NaN,NaN,NaN,기존,예ㆍ체능계열,76.0
13465,NaN,NaN,NaN,충북,NaN,한국교원대학교,제4대학,음악교육과,주간,NaN,NaN,NaN,NaN,기존,예ㆍ체능계열,77.0


In [260]:
df_intern_add.drop(columns = ['기준연도', '학교종류','설립구분','상태'], inplace= True)

In [261]:
import numpy as np

df_intern_add['현장실습 이수학생 비율'] = np.where(
    df_intern_add['학생정원'] == 0,
    np.nan,
    df_intern_add['현장실습 이수학생 수 곱셈'] / df_intern_add['학생정원']
)


In [262]:
# 원하는 컬럼 순서 지정
cols_order = [
    '지역', '학교', '단과대학', '계열', '학과', '구분', '학과상태', '학생정원',
    '실습기간 4주 이상 이수학생 수',
    '실습기간 8주 이상 이수학생 수',
    '실습기간 12주 이상 이수학생 수',
    '현장실습 이수학생 수 곱셈',
    '현장실습 이수학생 비율'
]

# 컬럼 순서 변경
df_intern_add = df_intern_add[cols_order]

# 학교 기준 정렬
df_intern_add = df_intern_add.sort_values(by='학교').reset_index(drop=True)


In [263]:
df_intern_add.drop(columns = ['실습기간 4주 이상 이수학생 수', '실습기간 8주 이상 이수학생 수','실습기간 12주 이상 이수학생 수'], inplace= True)

In [264]:
df_intern_add

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율
0,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN
1,경남,가야대학교(김해),단과대구분없음,자연과학계열,방사선학과,주간,기존,180.0,NaN,NaN
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN
4,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13462,경기,화성의과학대학교,단과대구분없음,자연과학계열,간호학과,주간,기존,160.0,NaN,NaN
13463,경기,화성의과학대학교,단과대구분없음,자연과학계열,ICT융합학과,주간,폐과,0.0,NaN,NaN
13464,경기,화성의과학대학교,단과대구분없음,자연과학계열,컴퓨터학과,주간,폐과,92.0,NaN,NaN
13465,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,뷰티아트케어학과,주간,변경,22.0,NaN,NaN


In [265]:
df_intern_add.dropna()

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율
19,경기,가천대학교,보건과학대학,자연과학계열,치위생학과,주간,기존,160.0,0.0,0.000000
20,경기,가천대학교,사회과학대학,인문ㆍ사회계열,경제학과,주간,기존,440.0,50.0,0.113636
21,경기,가천대학교,사회과학대학,인문ㆍ사회계열,관광경영학과,주간,기존,200.0,59.0,0.295000
27,경기,가천대학교,사회과학대학,인문ㆍ사회계열,미디어커뮤니케이션학과,주간,기존,240.0,64.0,0.266667
28,경기,가천대학교,사회과학대학,인문ㆍ사회계열,사회복지학과,주간,기존,200.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
13442,세종,홍익대학교 _제2캠퍼스,과학기술대학,공학계열,재료공학과,주간,폐과,92.0,5.0,0.054348
13445,세종,홍익대학교 _제2캠퍼스,과학기술대학,공학계열,건축공학부건축디자인전공(5년제),주간,기존,28.0,5.0,0.178571
13447,세종,홍익대학교 _제2캠퍼스,과학기술대학,공학계열,기계정보공학과,주간,기존,196.0,12.0,0.061224
13449,세종,홍익대학교 _제2캠퍼스,과학기술대학,공학계열,바이오화학공학과,주간,기존,180.0,6.0,0.033333


## 병합하기


In [266]:
#신입생 충원율 및 경쟁률, 현장실습 병합하기
#계열 데이터 병합 위해 현장실습 데이터 먼저 병합

In [267]:
#신입생 데이터 확인
df_freshman_selected.head()

,기준연도,학교,단과대학,학과,신입생 충원율(%),신입생 경쟁률
0,2023,가야대학교(김해),단과대구분없음,간호학과,100.0,9.0
1,2023,가야대학교(김해),단과대구분없음,물리치료학과,100.0,12.4
2,2023,가야대학교(김해),단과대구분없음,방사선학과,100.0,7.3
3,2023,가야대학교(김해),단과대구분없음,사회복지상담학과,50.0,4.4
4,2023,가야대학교(김해),단과대구분없음,스포츠재활복지학부,63.6,3.6


In [268]:
df_intern_add.head()

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율
0,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN
1,경남,가야대학교(김해),단과대구분없음,자연과학계열,방사선학과,주간,기존,180.0,NaN,NaN
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN
4,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,NaN,NaN


In [269]:
# 특수기호 제거 + 공백 제거 함수 / 텍스트 정리
def clean_text_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = df[col].str.replace('·', '', regex=False)
        df[col] = df[col].str.replace('・', '', regex=False)
        df[col] = df[col].str.replace('?', '', regex=False)
        df[col] = df[col].str.replace(r'\s*_(제\d+캠퍼스|분교)', r'_\1', regex=True)
        df[col] = df[col].str.strip()
    return df

In [270]:
# 전처리 적용
df_freshman_selected_and_clean = clean_text_columns(df_freshman_selected, ['학교', '학과', '기준연도','단과대학'])
df_intern_add_and_clean = clean_text_columns(df_intern_add, ['학교', '학과','단과대학'])

In [271]:
# 병합
df_merged_step_one = pd.merge(
    df_freshman_selected_and_clean,
    df_intern_add_and_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [272]:
df_merged_step_one.drop(columns = ['기준연도'], inplace=True)

In [273]:
# 원하는 컬럼 순서 지정
cols_order = [
    '지역', '학교', '단과대학', '계열', '학과', '구분', '학과상태', '학생정원',
    '신입생 충원율(%)','신입생 경쟁률',
    '현장실습 이수학생 수 곱셈',
    '현장실습 이수학생 비율'
]
# 컬럼 순서 변경
df_merged_step_one = df_merged_step_one[cols_order]

# 학교 기준 정렬
df_merged_step_one = df_merged_step_one.sort_values(by='학교').reset_index(drop=True)


In [274]:
df_merged_step_one

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율
0,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,100.0,9.0,NaN,NaN
1,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,특수교육과,주간,기존,114.0,88.9,2.9,NaN,NaN
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,커리어플러스학부사회복지상담학전공,주간,신설,20.0,NaN,NaN,NaN,NaN
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,"커리어플러스학부부동산금융재테크전공,헬스케어전공",주간,신설,30.0,NaN,NaN,NaN,NaN
4,NaN,가야대학교(김해),단과대구분없음,NaN,커리어플러스학부 사회복지상담학전공,NaN,NaN,NaN,65.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13867,경기,화성의과학대학교,단과대구분없음,자연과학계열,IT콘텐츠학과,주간,폐과,0.0,NaN,NaN,NaN,NaN
13868,경기,화성의과학대학교,단과대구분없음,자연과학계열,ICT융합학과,주간,폐과,0.0,NaN,NaN,NaN,NaN
13869,경기,화성의과학대학교,단과대구분없음,자연과학계열,컴퓨터사이언스학과,주간,변경,30.0,51.6,3.5,NaN,NaN
13870,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,뷰티아트케어학과,주간,변경,22.0,47.8,12.7,NaN,NaN


In [275]:
#중도탈락률
df_dropout_selected.head()

,기준연도,학교,단과대학,학과,중도탈락률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,4.5
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,16.7
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.9
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,8.7
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,5.6


In [276]:
df_dropout_selected_and_clean = clean_text_columns(df_dropout_selected, ['학교', '학과', '기준연도','단과대학'])

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\1628192743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\1628192743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace('·', '', regex=False)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\1628192743.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [277]:
df_dropout_selected_and_clean.drop(columns=['기준연도'], inplace= True)

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\3456081311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropout_selected_and_clean.drop(columns=['기준연도'], inplace= True)


In [278]:
# 병합
df_merged_step_two = pd.merge(
    df_merged_step_one,
    df_dropout_selected,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [279]:
df_merged_step_two

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율,중도탈락률(%)
0,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,100.0,9.0,NaN,NaN,4.5
1,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN,NaN,NaN,16.7
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN,NaN,NaN,31.9
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN,NaN,NaN,NaN
4,경남,가야대학교(김해),단과대구분없음,자연과학계열,귀금속주얼리학과,주간,폐과,50.0,NaN,NaN,NaN,NaN,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14506,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,스포츠건강학부,주간,변경,40.0,29.3,2.5,NaN,NaN,0.0
14507,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,스포츠레저학과,주간,폐과,140.0,NaN,NaN,NaN,NaN,26.1
14508,경기,화성의과학대학교,단과대구분없음,자연과학계열,의생명과학과,주간,변경,22.0,100.0,5.9,NaN,NaN,36.4
14509,경기,화성의과학대학교,단과대구분없음,자연과학계열,컴퓨터사이언스학과,주간,변경,30.0,51.6,3.5,NaN,NaN,26.7


In [280]:
#졸업생 취업률 및 진학률
df_gradu_final.head()

,기준연도,학교,단과대학,학과,졸업생 취업률(%),졸업생 진학률(%)
0,2023,가야대학교(김해),단과대구분없음,간호학과,85.9,0.0
1,2023,가야대학교(김해),단과대구분없음,경영물류학과,70.4,0.0
2,2023,가야대학교(김해),단과대구분없음,경찰소방학과,31.3,0.0
3,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,57.9,0.0
4,2023,가야대학교(김해),단과대구분없음,물리치료학과,88.7,0.0


In [281]:
df_gradu_final.drop(columns = ['기준연도'], inplace = True)

In [282]:
df_gradu_final_and_clean = clean_text_columns(df_gradu_final, ['학교','단과대학' , '학과'])

In [283]:
# 병합
df_merged_step_three = pd.merge(
    df_merged_step_two,
    df_gradu_final_and_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [284]:
df_merged_step_three

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%)
0,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,100.0,9.0,NaN,NaN,4.5,85.9,0.0
1,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN,NaN,NaN,16.7,70.4,0.0
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN,NaN,NaN,31.9,31.3,0.0
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,경남,가야대학교(김해),단과대구분없음,자연과학계열,귀금속주얼리학과,주간,폐과,50.0,NaN,NaN,NaN,NaN,8.7,57.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15140,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,스포츠건강학부,주간,변경,40.0,29.3,2.5,NaN,NaN,0.0,NaN,NaN
15141,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,스포츠레저학과,주간,폐과,140.0,NaN,NaN,NaN,NaN,26.1,100.0,12.5
15142,경기,화성의과학대학교,단과대구분없음,자연과학계열,의생명과학과,주간,변경,22.0,100.0,5.9,NaN,NaN,36.4,NaN,NaN
15143,경기,화성의과학대학교,단과대구분없음,자연과학계열,컴퓨터사이언스학과,주간,변경,30.0,51.6,3.5,NaN,NaN,26.7,NaN,NaN


In [285]:
#교원 1인당 연구실적
df_research_summary.head()

,기준연도,학교,단과대학,학과,전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적
0,2023,가야대학교(김해),단과대구분없음,방사선학과,0.7500,0.0000,0.1833
1,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,0.6667,0.0000,0.0000
2,2023,가야대학교(김해),단과대구분없음,교양학부,0.0000,0.0000,0.0000
3,2023,가야대학교(김해),단과대구분없음,간호학과,0.5446,0.0593,0.0867
4,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0000,0.0000,1.0000


In [286]:
df_research_summary.drop(columns = ['기준연도'], inplace = True)

In [287]:
df_research_summary_and_clean = clean_text_columns(df_research_summary, ['학교','단과대학' , '학과'])

In [288]:
# 병합
df_merged_step_four = pd.merge(
    df_merged_step_three,
    df_research_summary_and_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [289]:
df_merged_step_four

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적
0,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,100.0,9.0,NaN,NaN,4.5,85.9,0.0,0.5446,0.0593,0.0867
1,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN,NaN,NaN,16.7,70.4,0.0,0.3750,0.0000,0.0000
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN,NaN,NaN,31.9,31.3,0.0,0.6667,0.0000,0.0000
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,가야대학교(김해),단과대구분없음,NaN,교양학부,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16424,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,스포츠레저학과,주간,폐과,140.0,NaN,NaN,NaN,NaN,26.1,100.0,12.5,0.6500,0.3861,0.0278
16425,NaN,화성의과학대학교,단과대구분없음,NaN,의료심리학과,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1000,0.0000,0.0000
16426,경기,화성의과학대학교,단과대구분없음,자연과학계열,의생명과학과,주간,변경,22.0,100.0,5.9,NaN,NaN,36.4,NaN,NaN,0.0000,0.6416,0.0000
16427,경기,화성의과학대학교,단과대구분없음,자연과학계열,컴퓨터사이언스학과,주간,변경,30.0,51.6,3.5,NaN,NaN,26.7,NaN,NaN,NaN,NaN,NaN


In [290]:
#교원 1인당 연구비 수혜 실적
df_research_money_final.head()

,기준연도,학교,단과대학,학과,교내 1인당 연구비(천원),교외 1인당 연구비(천원)
0,2023,가야대학교(김해),단과대구분없음,방사선학과,1250.0,0.0
1,2023,가야대학교(김해),단과대구분없음,안경광학과,0.0,0.0
2,2023,가야대학교(김해),단과대구분없음,간호학과,900.0,0.0
3,2023,가야대학교(김해),단과대구분없음,교양학부,0.0,0.0
4,2023,가야대학교(김해),단과대구분없음,귀금속주얼리학과,1500.0,0.0


In [291]:
df_research_money_final.drop(columns=['기준연도'], inplace = True)

In [292]:
df_research_money_final_and_clean = clean_text_columns(df_research_money_final, ['학교','단과대학','학과'])

In [293]:
# 병합
df_merged_step_five = pd.merge(
    df_merged_step_four,
    df_research_money_final_and_clean,
    on=['학교', '학과','단과대학'],
    how='outer'
)

In [294]:
df_merged_step_five

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,현장실습 이수학생 수 곱셈,현장실습 이수학생 비율,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적,교내 1인당 연구비(천원),교외 1인당 연구비(천원)
0,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,100.0,9.0,NaN,NaN,4.5,85.9,0.0,0.5446,0.0593,0.0867,900.00,0.00
1,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN,NaN,NaN,16.7,70.4,0.0,0.3750,0.0000,0.0000,1125.00,0.00
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN,NaN,NaN,31.9,31.3,0.0,0.6667,0.0000,0.0000,3166.67,9943.33
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,가야대학교(김해),단과대구분없음,NaN,교양학부,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16527,경기,화성의과학대학교,단과대구분없음,예ㆍ체능계열,스포츠레저학과,주간,폐과,140.0,NaN,NaN,NaN,NaN,26.1,100.0,12.5,0.6500,0.3861,0.0278,0.00,2500.00
16528,NaN,화성의과학대학교,단과대구분없음,NaN,의료심리학과,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1000,0.0000,0.0000,0.00,0.00
16529,경기,화성의과학대학교,단과대구분없음,자연과학계열,의생명과학과,주간,변경,22.0,100.0,5.9,NaN,NaN,36.4,NaN,NaN,0.0000,0.6416,0.0000,0.00,173378.50
16530,경기,화성의과학대학교,단과대구분없음,자연과학계열,컴퓨터사이언스학과,주간,변경,30.0,51.6,3.5,NaN,NaN,26.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [295]:
#재학생 충원율
df_avg.head()

,연도,학교,계열,재학생충원율(%){C/(A-B)}X100,정원내 재학생 충원율(%){D/(A-B)}X100
0,2023,가야대학교(고령) _제2캠퍼스,공학계열,0.0,0.0
1,2023,가야대학교(고령) _제2캠퍼스,예ㆍ체능계열,0.0,0.0
2,2023,가야대학교(고령) _제2캠퍼스,의학계열,0.0,0.0
3,2023,가야대학교(고령) _제2캠퍼스,인문ㆍ사회계열,0.0,0.0
4,2023,가야대학교(고령) _제2캠퍼스,자연과학계열,0.0,0.0


In [296]:
df_avg.drop(columns=['연도'], inplace = True)

In [297]:
df_avg_and_clean = clean_text_columns(df_avg, ['학교'])

In [298]:
# 병합
df_merged_step_six = pd.merge(
    df_merged_step_five,
    df_avg_and_clean,
    on=['학교', '계열'],
    how='left'
)

In [304]:
# 대학이 같은 그룹에서 NaN인 지역 값을 채우기
df_merged_step_six['지역'] = df_merged_step_six.groupby('학교')['지역'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\1785105391.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged_step_six['지역'] = df_merged_step_six.groupby('학교')['지역'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [309]:
df_caps_avg_and_cleaned = clean_text_columns(df_caps_avg, ['학교','학과'])
df_merged_step_six = clean_text_columns(df_merged_step_six, ['학교','학과'])

In [310]:
# 병합
df_merged_step_seven = pd.merge(
    df_merged_step_six,
    df_caps_avg_and_cleaned,
    on=['학교', '학과'],
    how='left'
)

In [ ]:
# df_merged_step_seven.to_excel('09176페이지 전체 대학 데이터.xlsx', index=False)

## 강릉원주대 필터링

In [170]:
#강릉원주대 데이터
df_gw = df_merged_step_six[df_merged_step_six['학교'].str.contains('강릉원주대')]

In [171]:
df_gw.head()

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,...,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적,교내 1인당 연구비(천원),교외 1인당 연구비(천원),재학생충원율(%){C/(A-B)}X100,정원내 재학생 충원율(%){D/(A-B)}X100
3683,강원,국립강릉원주대학교,공과대학,공학계열,건설환경공학과,주간,신설,37.0,100.0,6.0,...,4.7,NaN,NaN,0.719,1.2851,0.0,0.0,263912.75,91.95,86.0
3684,강원,국립강릉원주대학교,사회과학대학,인문ㆍ사회계열,경영학과,주간,기존,125.0,100.0,7.2,...,6.1,66.7,0.0,0.960,0.5000,0.0,720.0,3720.00,98.50,85.6
3685,강원,국립강릉원주대학교,사회과학대학,인문ㆍ사회계열,경제학과,주간,기존,113.0,100.0,3.8,...,5.6,73.7,8.7,1.350,0.0500,0.0,320.0,0.00,98.50,85.6
3686,강원,국립강릉원주대학교,예술체육대학,NaN,공예조형디자인학과,NaN,NaN,NaN,NaN,NaN,...,NaN,33.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3687,강원,국립강릉원주대학교,사회과학대학,인문ㆍ사회계열,관광경영학과,주간,기존,173.0,97.7,6.9,...,3.0,61.7,0.0,0.825,0.3417,0.0,487.5,12945.00,98.50,85.6


In [172]:
import pandas as pd
#엑셀 파일 불러오기 캡스톤디자인용
def load_and_clean_data_for_cap(filepath):
    df = pd.read_excel(filepath, header = [3,4,5])

    new_columns = []
    for col in df.columns:
        merged_name = [str(c) for c in col if 'Unnamed' not in str(c)]
        new_columns.append(' '.join(merged_name).strip())

    df.columns = new_columns
    df= df.fillna(method='ffill', axis=0)

    return df

In [173]:
df_gw_cap = load_and_clean_data_for_cap(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\캡스톤디자인\캡스톤디자인 학교별 학과 데이터\캡스톤 디자인(창의적 설계) 운영 현황 (대학)_국립강릉원주대학교_기준연도 2023.xlsx")
df_gw_cap2 = load_and_clean_data_for_cap(r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\데이터정리\학과경쟁력분석\캡스톤디자인\캡스톤디자인 학교별 학과 데이터\캡스톤 디자인(창의적 설계) 운영 현황 (대학)_국립강릉원주대학교_제2캠퍼스_기준연도 2023.xlsx")

C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\1248805691.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)
C:\Users\윤서현\AppData\Local\Temp\ipykernel_12428\1248805691.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method='ffill', axis=0)


In [174]:
df_gw_cap.head()

,기준연도,학교,단과대학,학과\n(전공),계열,구분,교과목명,학기,학점수,이수\n학생수(명) 해당학과,이수\n학생수(명) 타학과 계열 인문·사회,이수\n학생수(명) 타학과 계열 자연과학,이수\n학생수(명) 타학과 계열 공학,이수\n학생수(명) 타학과 계열 의학,이수\n학생수(명) 타학과 계열 예·체능,지원금\n수령\n학생수(명),지원금 수령\n총금액(원) 시제(작)품\n제작비,지원금 수령\n총금액(원) 교육프로그램\n지원비
0,합 계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171,1424,29,9,0,0,1,922,79589890,61547170
1,2023,국립강릉원주대학교,공과대학,건설환경공학과,공학계열,주간,-,1학기,0,0,0,0,0,0,0,0,0,0
2,2023,국립강릉원주대학교,공과대학,건설환경공학과,공학계열,주간,종합설계_캡스톤디자인,2학기,1,35,0,0,0,0,0,32,2650000,1891500
3,2023,국립강릉원주대학교,공과대학,건설환경공학과,공학계열,주야간구분없음,-,1학기,0,0,0,0,0,0,0,0,0,0
4,2023,국립강릉원주대학교,공과대학,건설환경공학과,공학계열,주야간구분없음,-,2학기,0,0,0,0,0,0,0,0,0,0


In [175]:
# '이수\n학생수(명) 타학과 계열 인문·사회', '이수\n학생수(명) 타학과 계열 자연과학','이수\n학생수(명) 타학과 계열 공학', '이수\n학생수(명) 타학과 계열 의학','이수\n학생수(명) 타학과 계열 예·체능'
df_gw_cap['타계열학생'] = (df_gw_cap['이수\n학생수(명) 타학과 계열 인문·사회'] + df_gw_cap['이수\n학생수(명) 타학과 계열 자연과학'] + df_gw_cap['이수\n학생수(명) 타학과 계열 공학'] 
                    + df_gw_cap['이수\n학생수(명) 타학과 계열 의학'] + df_gw_cap['이수\n학생수(명) 타학과 계열 예·체능'])

df_gw_cap2['타계열학생'] = (df_gw_cap2['이수\n학생수(명) 타학과 계열 인문·사회'] + df_gw_cap2['이수\n학생수(명) 타학과 계열 자연과학'] + df_gw_cap2['이수\n학생수(명) 타학과 계열 공학'] 
                    + df_gw_cap2['이수\n학생수(명) 타학과 계열 의학'] + df_gw_cap2['이수\n학생수(명) 타학과 계열 예·체능'])

In [176]:
def create_summarized_dataframe(df):
    # 필요한 컬럼만 선택
    summary_df = df[['기준연도', '학교', '단과대학', '학과\n(전공)', '계열','구분','교과목명','학기','이수\n학생수(명) 해당학과','타계열학생']].copy()
    return summary_df

df_gw_cap_filtered = create_summarized_dataframe(df_gw_cap)
df_gw_cap2_filtered = create_summarized_dataframe(df_gw_cap2)

In [177]:
df_gw_cap_noon = df_gw_cap_filtered[df_gw_cap_filtered['구분'] != '야간']
df_gw_cap2_noon = df_gw_cap2_filtered[df_gw_cap2_filtered['구분'] != '야간']

In [178]:
df_gw_cap_noon = pd.concat([df_gw_cap_noon,df_gw_cap2_noon])

In [179]:
df_cap_avg = df_gw_cap_noon.groupby(['학교', '단과대학','학과\n(전공)']).agg({
        '이수\n학생수(명) 해당학과': 'mean',
        '타계열학생': 'mean'
    }).reset_index()

# 컬럼명 변경
df_cap_avg = df_cap_avg.rename(columns={
        '이수\n학생수(명) 해당학과': '해당학과 평균 이수학생수',
        '타계열학생': '타계열 평균 이수학생수'})

#반올림
df_cap_avg[['해당학과 평균 이수학생수', '타계열 평균 이수학생수']] = df_cap_avg[[
    '해당학과 평균 이수학생수', '타계열 평균 이수학생수']].round(2)


In [180]:
df_cap_avg['캡스톤디자인 평균 이수 학생 수'] = df_cap_avg['해당학과 평균 이수학생수'] + df_cap_avg['타계열 평균 이수학생수']

In [181]:
df_cap_avg.columns

Index(['학교', '단과대학', '학과\n(전공)', '해당학과 평균 이수학생수', '타계열 평균 이수학생수',
       '캡스톤디자인 평균 이수 학생 수'],
      dtype='object')

In [182]:
df_cap_final = df_cap_avg[['학교', '단과대학', '학과\n(전공)','캡스톤디자인 평균 이수 학생 수']].copy()

In [183]:
df_merged_step_six

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,...,중도탈락률(%),졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적,교내 1인당 연구비(천원),교외 1인당 연구비(천원),재학생충원율(%){C/(A-B)}X100,정원내 재학생 충원율(%){D/(A-B)}X100
0,경남,가야대학교(김해),단과대구분없음,자연과학계열,간호학과,주간,기존,602.0,100.0,9.0,...,4.5,85.9,0.0,0.5446,0.0593,0.0867,900.00,0.00,89.95,78.80
1,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경영물류학과,주간,폐과,65.0,NaN,NaN,...,16.7,70.4,0.0,0.3750,0.0000,0.0000,1125.00,0.00,68.95,68.35
2,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰소방학과,주간,폐과,30.0,NaN,NaN,...,31.9,31.3,0.0,0.6667,0.0000,0.0000,3166.67,9943.33,68.95,68.35
3,경남,가야대학교(김해),단과대구분없음,인문ㆍ사회계열,경찰행정학과,주간,폐과,80.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.95,68.35
4,경남,가야대학교(김해),단과대구분없음,NaN,교양학부,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.00,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18578,부산,화신사이버대학교,단과대구분없음,NaN,스마트팜농수산복지학과,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.00,0.00,NaN,NaN
18579,부산,화신사이버대학교,단과대구분없음,NaN,실용외국어학과,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.00,0.00,NaN,NaN
18580,부산,화신사이버대학교,단과대구분없음,NaN,영상콘텐츠학과,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.00,0.00,NaN,NaN
18581,부산,화신사이버대학교,단과대구분없음,NaN,융합스포츠지도학과,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.00,0.00,NaN,NaN


In [184]:
df_cap_final.rename(columns={'학과\n(전공)' : '학과'}, inplace=True)

In [185]:
df_cap_final_and_clean = clean_text_columns(df_cap_final, ['학교','학과','단과대학'])

In [186]:
# 병합
df_gw_total = pd.merge(
    df_gw,
    df_cap_final_and_clean,
    on=['학교', '단과대학','학과'],
    how='left'
)

In [187]:
df_gw_total

,지역,학교,단과대학,계열,학과,구분,학과상태,학생정원,신입생 충원율(%),신입생 경쟁률,...,졸업생 취업률(%),졸업생 진학률(%),전임교원 1인당 논문 실적 국내,전임교원 1인당 논문 실적 국제,전임교원 1인당 저∙역서 실적,교내 1인당 연구비(천원),교외 1인당 연구비(천원),재학생충원율(%){C/(A-B)}X100,정원내 재학생 충원율(%){D/(A-B)}X100,캡스톤디자인 평균 이수 학생 수
0,강원,국립강릉원주대학교,공과대학,공학계열,건설환경공학과,주간,신설,37.0,100.0,6.0,...,NaN,NaN,0.7190,1.2851,0.0000,0.00,263912.75,91.95,86.00,8.75
1,강원,국립강릉원주대학교,사회과학대학,인문ㆍ사회계열,경영학과,주간,기존,125.0,100.0,7.2,...,66.7,0.00,0.9600,0.5000,0.0000,720.00,3720.00,98.50,85.60,17.29
2,강원,국립강릉원주대학교,사회과학대학,인문ㆍ사회계열,경제학과,주간,기존,113.0,100.0,3.8,...,73.7,8.70,1.3500,0.0500,0.0000,320.00,0.00,98.50,85.60,4.25
3,강원,국립강릉원주대학교,예술체육대학,NaN,공예조형디자인학과,NaN,NaN,NaN,NaN,NaN,...,33.3,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,강원,국립강릉원주대학교,사회과학대학,인문ㆍ사회계열,관광경영학과,주간,기존,173.0,97.7,6.9,...,61.7,0.00,0.8250,0.3417,0.0000,487.50,12945.00,98.50,85.60,9.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,강원,국립강릉원주대학교_제2캠퍼스,보건복지대학,인문ㆍ사회계열,유아교육과,주간,기존,240.0,100.0,5.9,...,46.9,0.00,1.4466,0.0000,0.0741,2408.89,0.00,102.60,98.65,0.00
76,강원,국립강릉원주대학교_제2캠퍼스,과학기술대학,공학계열,자동차공학과,주간,기존,131.0,100.0,6.1,...,64.3,9.68,0.8100,0.1382,0.0000,900.00,40841.20,103.00,95.70,11.75
77,강원,국립강릉원주대학교_제2캠퍼스,과학기술대학,공학계열,전기공학과,주간,기존,130.0,100.0,5.6,...,77.8,3.45,1.2300,0.7500,0.0000,1200.00,36900.00,103.00,95.70,5.00
78,강원,국립강릉원주대학교_제2캠퍼스,과학기술대학,공학계열,정보통신공학과,주간,기존,121.0,100.0,4.4,...,69.0,6.45,0.3600,0.0000,0.9999,0.00,28368.60,103.00,95.70,11.75


In [188]:
df_gw_total.to_excel('08086페이지강릉원주대.xlsx', index = False)